# Client Test

In [9]:
import os
import sys
import time
 
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'client')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'node')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'shared')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'micropython')))
 
import client
from collections import OrderedDict

### Start client

In [10]:
the_client = client.Client()
the_client.start()

My name is client_366

[connected: ('192.168.0.105', 9662)]
Sending 305 bytes
Message:
OrderedDict([('command', 'set connection name'), ('correlation_id', '2017-01-27 09:55:34.908300'), ('kwargs', {'name': 'client_366'}), ('message_id', '2017-01-27 09:55:34.908300'), ('need_result', True), ('receiver', 'Hub'), ('reply_to', 'client_366'), ('sender', 'client_366'), ('type', 'command')])

[Listen to messages]


### Prepare messages

In [11]:
messages = OrderedDict()

messages['read_GPIOs'] = {'type': 'command',
                          'command': 'read GPIOs',
                          'need_result': True}

messages['blink_led'] = {'type': 'command',
                         'command': 'blink led',
                         'kwargs': {'times': 10, 'forever': False, 'on_seconds': 0.1, 'off_seconds': 0.1}} 

In [12]:
# messages['write_GPIOs'] = {'type': 'command',
                           # 'command': 'write GPIOs',
                           # 'kwargs': {'pins_and_values': [(2, 0), (2, 1), (2, 0),]}} 

# messages['test eval'] = {'type': 'eval',
                         # 'to_evaluate': '2+3',
                         # 'need_result': True}                                   

# messages['test exec'] = {'type': 'exec',
                         # 'to_exec': 'print("Testing exec !")'}

# with open('script_to_deploy.py') as f:
    # script = f.read()        
# messages['test upload script'] = {'type': 'script', 
                                  # 'script': script} 


Data received: 401 bytes
Message:
OrderedDict([('correlation_id', '2017-01-27 09:55:34.908300'), ('message_id', '2017-01-27 09:55:34.986300'), ('receiver', 'client_366'), ('reply_to', 'Hub'), ('result', {'NodeMCU_1dsc000': "('192.168.0.133', 17140)", 'NodeMCU_f1d30800': "('192.168.0.116', 28082)", 'client_366': "('192.168.0.105', 51462)"}), ('sender', 'Hub'), ('time_stamp', '2017-01-27 09:55:34.986300'), ('type', 'result')])



### Ask Hub for a list of connected nodes

In [13]:
message = {'type': 'command',
                   'command': 'list connections by name',
                   'need_result': True}     

the_message, asynch_result = the_client.request('Hub', message) 

try:
    remote_nodes = list(asynch_result.get().keys())

    print ('\n[____________ Connected nodes ____________]\n')        
    print('\nConnected nodes:\n{}\n'.format(remote_nodes)) 
    
except Exception as e:
    print(e)

Sending 276 bytes
Message:
OrderedDict([('command', 'list connections by name'), ('correlation_id', '2017-01-27 09:55:35.341300'), ('message_id', '2017-01-27 09:55:35.341300'), ('need_result', True), ('receiver', 'Hub'), ('reply_to', 'client_366'), ('sender', 'client_366'), ('type', 'command')])


Data received: 401 bytes
Message:
OrderedDict([('correlation_id', '2017-01-27 09:55:35.341300'), ('message_id', '2017-01-27 09:55:35.421300'), ('receiver', 'client_366'), ('reply_to', 'Hub'), ('result', {'NodeMCU_1dsc000': "('192.168.0.133', 17140)", 'NodeMCU_f1d30800': "('192.168.0.116', 28082)", 'client_366': "('192.168.0.105', 51462)"}), ('sender', 'Hub'), ('time_stamp', '2017-01-27 09:55:35.421300'), ('type', 'result')])


[____________ Connected nodes ____________]


Connected nodes:
['NodeMCU_1dsc000', 'NodeMCU_f1d30800', 'client_366']



### Send out messages and get asynchonous results

In [14]:
print ('\n[______________ Sending messages ______________]\n')
                
results = []

## send out the messages
for message in messages.values():
    for remote_node in remote_nodes:
        if remote_node != the_client.node.worker.name:  # exclude client self
            formatted_message, asynch_result = the_client.request(remote_node, message) 
            results.append((formatted_message, asynch_result))


[______________ Sending messages ______________]

Sending 274 bytes
Message:
OrderedDict([('command', 'read GPIOs'), ('correlation_id', '2017-01-27 09:55:35.531900'), ('message_id', '2017-01-27 09:55:35.531900'), ('need_result', True), ('receiver', 'NodeMCU_1dsc000'), ('reply_to', 'client_366'), ('sender', 'client_366'), ('type', 'command')])

Sending 275 bytes
Message:
OrderedDict([('command', 'read GPIOs'), ('correlation_id', '2017-01-27 09:55:35.569900'), ('message_id', '2017-01-27 09:55:35.569900'), ('need_result', True), ('receiver', 'NodeMCU_f1d30800'), ('reply_to', 'client_366'), ('sender', 'client_366'), ('type', 'command')])

Sending 334 bytes
Message:
OrderedDict([('command', 'blink led'), ('correlation_id', '2017-01-27 09:55:35.607500'), ('kwargs', {'forever': False, 'times': 10, 'on_seconds': 0.1, 'off_seconds': 0.1}), ('message_id', '2017-01-27 09:55:35.607500'), ('receiver', 'NodeMCU_1dsc000'), ('reply_to', 'client_366'), ('sender', 'client_366'), ('type', 'command')])



### Actually get the results

In [15]:
# collect and print results        
print('\n[_________ Wait few seconds for reply _________]\n')

for (message, result) in results:
    try:
        if message.get('need_result'):
            print('\n[Result for request]:\n___Request___:\n{0}\n___Result____:\n{1}\n'.format(message, result.get() if result else None))
    except Exception as e:
        print('\n[{}]\nMessage:\n{}'.format(e, message))


[_________ Wait few seconds for reply _________]


Data received: 322 bytes
Message:
OrderedDict([('correlation_id', '2017-01-27 09:55:35.531900'), ('message_id', '6961703'), ('receiver', 'client_366'), ('reply_to', 'NodeMCU_1dsc000'), ('result', [[5, 1], [12, 1], [13, 1], [14, 1], [15, 0], [16, 0]]), ('sender', 'NodeMCU_1dsc000'), ('time_stamp', '2017-01-27 09:55:35.908700'), ('type', 'result')])


[Result for request]:
___Request___:
{'receiver': 'NodeMCU_1dsc000', 'message_id': '2017-01-27 09:55:35.531900', 'result': None, 'need_result': True, 'correlation_id': '2017-01-27 09:55:35.531900', 'type': 'command', 'reply_to': 'client_366', 'command': 'read GPIOs', 'sender': 'client_366'}
___Result____:
[[5, 1], [12, 1], [13, 1], [14, 1], [15, 0], [16, 0]]


Data received: 322 bytes
Message:
OrderedDict([('correlation_id', '2017-01-27 09:55:35.569900'), ('message_id', '29265'), ('receiver', 'client_366'), ('reply_to', 'NodeMCU_f1d30800'), ('result', [[5, 0], [12, 1], [13, 1], [14, 1], [1

### Stop the demo

In [16]:
# Stopping
the_client.stop()
the_client = None
print ('\n[________________ Demo stopped ________________]\n')


[________________ Demo stopped ________________]

